In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import pandas as pd
import random

### Get sequence around m6A sites 

In [2]:
Data = pd.read_csv("m6A_for_CodonEnumeration_Top100K.csv", keep_default_na=False)

In [3]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,...,m6AToStopCodon,PreSequence,LastExonSequence,IntervalSequence,CodonSequence,NoncodingSequence,Protein,StopCodonCount,Motif,m6ASequence
0,Cdh20,chr1,+,104768528,104995481,104934096,104994385,104993879,104995481,104994284,...,100,AGAAAACGCGCGAAGGAGGGGAAGAAGCGGACCGCGCGCTGGTCTT...,TGTTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACC...,TG,TTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACCCT...,CAGAAGCCCGGAGGCAAAGGGGCACCCAAATCCAGACGTTCTCAAA...,LVLLILSMRRHRKQPYIIDDDENIHENIVRYDDEGGGEEDTEAFDI...,1,GGACT,GAC
1,Cdh7,chr1,+,109983736,110139001,109994179,110138355,110137861,110139001,110138138,...,216,TTGGACATCTTCGTGGGTTCACTTGGCACAAGGCTAGTTGCTGGTA...,TGCTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCC...,TG,CTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCCTC...,CCTTGGACCCTTACTTTGGAATGCACTGCATAGAAAGTCAGCAGGA...,LILLIVTMRRRKKEPLIFDEERDIRENIVRYDDEGGGEEDTEAFDM...,1,AGACT,GAC
2,Brinp3,chr1,+,146494759,146902471,146514717,146902117,146901000,146902471,146901641,...,475,GACAGTAGATTTCCGAGAGGAAAGGAGAAGAGAAGATAGGGATGGA...,AACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGC...,A,ACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGCA...,CCATAAATGTCAAGCACAACCCAAAATCTTGAAGGAGTTTTTACAG...,TSTYWLTRIQSFLYCNENGLLGSFSEETHSCTCPNDQVVCTAFLPC...,1,TAACT,AAC
3,Lgsn,chr1,+,31176400,31204725,31176450,31204530,31203330,31204725,31204434,...,95,GCTGAAGGGAGTGTGGATGCCACGGCTGATTTATGTAAGAAACAGT...,GAAAAAGTGATCCACGGAGTTTTCATGCCCCGTGGGTATCTGGAAT...,,GAAAAAGTGATCCACGGAGTTTTCATGCCCCGTGGGTATCTGGAAT...,GATGGGACTCTGCCCTGTAGAGATATGATGAGTAGCTAGTTTATCA...,EKVIHGVFMPRGYLELMPNPKDNEVNHIRATCFNSDIVLMPELSTF...,1,AGACT,GAC
4,Ercc5,chr1,+,44147846,44181260,44148062,44181082,44180530,44181260,44180987,...,94,ACATTTAAATTCCGCTGTTCTTTCGGGAGACCGGGTCCGCTCCGGA...,ACCCAGCTCCGGATTGATTCCTTTTTTAGACTAGCTCAGCAGGAAA...,,ACCCAGCTCCGGATTGATTCCTTTTTTAGACTAGCTCAGCAGGAAA...,TTTAAAGAGAAAAAAAAAAGTCATTTAGAAACAGAAAGTGCTTACA...,TQLRIDSFFRLAQQEKQDAKLIKSHRLNRAVTCILRKEREEKAPEL...,1,GGACA,GAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,Ifit1bl1,chr19,-,34592887,34601968,34593642,34601921,34592887,34595023,34594067,...,425,ACTTTTGATTGAGTTTGGAAGCATCTCTTGGCCTGGATTAAAGTGC...,TGCAAAATCTCATAGTTGTCTCATCTATGACAGCCTGGTTGAGCTG...,T,GCAAAATCTCATAGTTGTCTCATCTATGACAGCCTGGTTGAGCTGA...,GGAAGAGCTTCTTGAGTGGCTTGTTAAGCGTGAGCATGACTTTCTA...,AKSHSCLIYDSLVELRCHFTWKLVIEKVDMPDLEVRISETEFFDAS...,1,GGACT,GAC
951,Ifit1bl2,chr19,-,34617050,34640743,34618813,34629759,34617050,34620182,34619238,...,425,CTGTGGGGTTTGCCCTACTGCATTCTCCACTGAAATCGATGAGGAG...,TGAAGAATCTCATAAAAGTCATATACATGACAGCCTGGATGAGCTG...,T,GAAGAATCTCATAAAAGTCATATACATGACAGCCTGGATGAGCTGA...,GGTAGAGCTTCTTGTTAAGCGTGAGCATGACTTCCTATTCTACTAG...,EESHKSHIHDSLDELRCHFTWELDIKDKHIHDLEIKISETEFRDPI...,1,AGACT,GAC
952,Ppp1r3c,chr19,-,36731736,36736653,36733414,36736566,36731736,36734354,36733577,...,163,CTGGACCGCGGCGCTGCATCCTCTTAAGTACTGAGTGCGAAGTTGC...,AATGATCCATGTGCTAGATCCACGTCCTTTGACAAGTTCCGTCATG...,A,ATGATCCATGTGCTAGATCCACGTCCTTTGACAAGTTCCGTCATGC...,GTTAGGCAATCTACACACTGCCCTTGTCTGATCATAATCCTCATGC...,MIHVLDPRPLTSSVMPVDMAMRICLAHSPPLKSFLGPYNGFQRRNF...,1,TGACG,GAC
953,Slc25a28,chr19,-,43663821,43674881,43664125,43674766,43663821,43664643,43664372,...,247,CCGGAGGCCCCGGGCTCTCTGGGCCGCGCCTGAGGCGGACGCAACG...,TCGTCAAGCAGAGGATGCAGATGTACAACTCGCCGTACCACCGCGT...,TC,GTCAAGCAGAGGATGCAGATGTACAACTCGCCGTACCACCGCGTGA...,AGGGATGCTGAGTGGACCAGGGTCCGACCGCACGGCTGCATCCTGC...,VKQRMQMYNSPYHRVTDCVRAVWQNEGAGAFYRSYTTQLTMNVPFQ...,1,GGACG,GAC


### Define m6A sites in Codon

In [4]:
Data["m6AInterval"] = Data["m6AToCodon"]%3

In [5]:
Data["m6AToCodon"] = Data["m6APosition"] - Data["Interval"]

In [6]:
Data = Data[(Data["m6AToCodon"]>=51)&(Data["m6AToStopCodon"]>=51)]
Data = Data.reset_index(drop = True)

In [7]:
pd.value_counts(Data["m6ASequence"])

GAC    796
AAC    159
Name: m6ASequence, dtype: int64

In [8]:
Motif = Data

In [9]:
for i in range(len(Motif)):
    Motif.loc[i,("m6ASiteBeforeCodonSwap")] = Motif.loc[i,("LastExonSequence")][(Motif.loc[i,("m6APosition")]-50):(Motif.loc[i,("m6APosition")]+52)]

In [10]:
#for i in range(len(Motif)):
#    if Motif.loc[i,("m6AInterval")] == 0:
#        Motif.loc[i,("m6ASiteBeforeCodonSwap")] = Motif.loc[i,("CodonSequence")][(Motif.loc[i,("m6AToCodon")]-3-3-42):(Motif.loc[i,("m6AToCodon")]+6+42)]
#    elif Motif.loc[i,("m6AInterval")] == 1:
#        Motif.loc[i,("m6ASiteBeforeCodonSwap")] = Motif.loc[i,("CodonSequence")][(Motif.loc[i,("m6AToCodon")]-4-3-42):(Motif.loc[i,("m6AToCodon")]+5+42)]
#    elif Motif.loc[i,("m6AInterval")] == 2:
#        Motif.loc[i,("m6ASiteBeforeCodonSwap")] = Motif.loc[i,("CodonSequence")][(Motif.loc[i,("m6AToCodon")]-5-42):(Motif.loc[i,("m6AToCodon")]+7+42)]

In [11]:
Motif

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,...,LastExonSequence,IntervalSequence,CodonSequence,NoncodingSequence,Protein,StopCodonCount,Motif,m6ASequence,m6AInterval,m6ASiteBeforeCodonSwap
0,Cdh20,chr1,+,104768528,104995481,104934096,104994385,104993879,104995481,104994284,...,TGTTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACC...,TG,TTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACCCT...,CAGAAGCCCGGAGGCAAAGGGGCACCCAAATCCAGACGTTCTCAAA...,LVLLILSMRRHRKQPYIIDDDENIHENIVRYDDEGGGEEDTEAFDI...,1,GGACT,GAC,1,GGACGGCTCTGTGGCCGGCTCGCTCAGCTCCCTGCAGTCAGCCACC...
1,Cdh7,chr1,+,109983736,110139001,109994179,110138355,110137861,110139001,110138138,...,TGCTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCC...,TG,CTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCCTC...,CCTTGGACCCTTACTTTGGAATGCACTGCATAGAAAGTCAGCAGGA...,LILLIVTMRRRKKEPLIFDEERDIRENIVRYDDEGGGEEDTEAFDM...,1,AGACT,GAC,2,TTTAAAAACATCCCAGATAATGTCATTTTTAGGGAATTTATTTGGG...
2,Brinp3,chr1,+,146494759,146902471,146514717,146902117,146901000,146902471,146901641,...,AACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGC...,A,ACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGCA...,CCATAAATGTCAAGCACAACCCAAAATCTTGAAGGAGTTTTTACAG...,TSTYWLTRIQSFLYCNENGLLGSFSEETHSCTCPNDQVVCTAFLPC...,1,TAACT,AAC,1,CTTTCCAGACTGGGAGCGGACTAAGCTGGACCTCCCTCTACAGTGC...
3,Lgsn,chr1,+,31176400,31204725,31176450,31204530,31203330,31204725,31204434,...,GAAAAAGTGATCCACGGAGTTTTCATGCCCCGTGGGTATCTGGAAT...,,GAAAAAGTGATCCACGGAGTTTTCATGCCCCGTGGGTATCTGGAAT...,GATGGGACTCTGCCCTGTAGAGATATGATGAGTAGCTAGTTTATCA...,EKVIHGVFMPRGYLELMPNPKDNEVNHIRATCFNSDIVLMPELSTF...,1,AGACT,GAC,0,CCCTTGCCGCCCTGGAGCAAGATGAGTGTCTGAAGCAGGCTCTGGG...
4,Ercc5,chr1,+,44147846,44181260,44148062,44181082,44180530,44181260,44180987,...,ACCCAGCTCCGGATTGATTCCTTTTTTAGACTAGCTCAGCAGGAAA...,,ACCCAGCTCCGGATTGATTCCTTTTTTAGACTAGCTCAGCAGGAAA...,TTTAAAGAGAAAAAAAAAAGTCATTTAGAAACAGAAAGTGCTTACA...,TQLRIDSFFRLAQQEKQDAKLIKSHRLNRAVTCILRKEREEKAPEL...,1,GGACA,GAC,1,TGGGAGGCAAGGCTGTGTGTCCACTAGTAGCTCCAGCGAGGATGAT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,Ifit1bl1,chr19,-,34592887,34601968,34593642,34601921,34592887,34595023,34594067,...,TGCAAAATCTCATAGTTGTCTCATCTATGACAGCCTGGTTGAGCTG...,T,GCAAAATCTCATAGTTGTCTCATCTATGACAGCCTGGTTGAGCTGA...,GGAAGAGCTTCTTGAGTGGCTTGTTAAGCGTGAGCATGACTTTCTA...,AKSHSCLIYDSLVELRCHFTWKLVIEKVDMPDLEVRISETEFFDAS...,1,GGACT,GAC,0,GAAAAAGAGCAGACCAATCCATCCATTTGGCTATATGTCATTTTAA...
951,Ifit1bl2,chr19,-,34617050,34640743,34618813,34629759,34617050,34620182,34619238,...,TGAAGAATCTCATAAAAGTCATATACATGACAGCCTGGATGAGCTG...,T,GAAGAATCTCATAAAAGTCATATACATGACAGCCTGGATGAGCTGA...,GGTAGAGCTTCTTGTTAAGCGTGAGCATGACTTCCTATTCTACTAG...,EESHKSHIHDSLDELRCHFTWELDIKDKHIHDLEIKISETEFRDPI...,1,AGACT,GAC,0,GAAAAAGAGCAGACCAATCATGCCATTTGGCTATATGTCATTTTAA...
952,Ppp1r3c,chr19,-,36731736,36736653,36733414,36736566,36731736,36734354,36733577,...,AATGATCCATGTGCTAGATCCACGTCCTTTGACAAGTTCCGTCATG...,A,ATGATCCATGTGCTAGATCCACGTCCTTTGACAAGTTCCGTCATGC...,GTTAGGCAATCTACACACTGCCCTTGTCTGATCATAATCCTCATGC...,MIHVLDPRPLTSSVMPVDMAMRICLAHSPPLKSFLGPYNGFQRRNF...,1,TGACG,GAC,1,CAACAATGAGGGTCAGAATTACAGAATTGTCCATGTGCAATGGAAA...
953,Slc25a28,chr19,-,43663821,43674881,43664125,43674766,43663821,43664643,43664372,...,TCGTCAAGCAGAGGATGCAGATGTACAACTCGCCGTACCACCGCGT...,TC,GTCAAGCAGAGGATGCAGATGTACAACTCGCCGTACCACCGCGTGA...,AGGGATGCTGAGTGGACCAGGGTCCGACCGCACGGCTGCATCCTGC...,VKQRMQMYNSPYHRVTDCVRAVWQNEGAGAFYRSYTTQLTMNVPFQ...,1,GGACG,GAC,1,CTGTGGGGCCTGCGCAGGAGCAGTAGCTGCCGCCGCCACAACCCCA...


In [12]:
Sequence = Motif["m6ASiteBeforeCodonSwap"].tolist()
df = pd.DataFrame({"Index":range(0, 101-2)})
for i in range(len(Motif)):
    seq = Sequence[i]
    seq= seq.upper()
    New = []
    if len(seq)%3 == 0:
        for j in range(0, 99, 1):
            string = seq[j:j+3]
            New.append(string)
    New = np.array(New)
    New = pd.DataFrame({Data.loc[i,"name"]:New})
    df = pd.merge(df, New, how="outer", left_index=True, right_index=True)

In [13]:
df

,Index,Cdh20,Cdh7,Brinp3,Lgsn,Ercc5,Zdbf2,Gin1,Cntnap5b,Edem3,...,Plgrkt,9930021J03Rik,Sac3d1,Zfp91,Cep78,Ifit1bl1,Ifit1bl2,Ppp1r3c,Slc25a28,Pdzd8
0,0,GGA,TTT,CTT,CCC,TGG,AGT,CAG,GCA,TGG,...,GCT,GAT,CTG,CTT,GTG,GAA,GAA,CAA,CTG,CGC
1,1,GAC,TTA,TTT,CCT,GGG,GTA,AGG,CAA,GGG,...,CTG,ATA,TGG,TTC,TGA,AAA,AAA,AAC,TGT,GCA
2,2,ACG,TAA,TTC,CTT,GGA,TAT,GGC,AAA,GGT,...,TGA,TAA,GGC,TCT,GAA,AAA,AAA,ACA,GTG,CAG
3,3,CGG,AAA,TCC,TTG,GAG,ATG,GCA,AAA,GTC,...,GAG,AAT,GCC,CTA,AAT,AAA,AAA,CAA,TGG,AGA
4,4,GGC,AAA,CCA,TGC,AGG,TGT,CAA,AAG,TCC,...,AGG,ATC,CCC,TAC,ATC,AAG,AAG,AAT,GGG,GAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,TTC,CTC,ACA,AAA,AAG,CAG,ATT,ACA,TCC,...,GTA,ACT,ACT,ATT,ACA,GGC,GGC,CAG,AAC,TCT
95,95,TCC,TCT,CAG,AAC,AGA,AGG,TTC,CAG,CCA,...,TAA,CTC,CTC,TTG,CAT,GCT,GCT,AGG,ACT,CTG
96,96,CCG,CTT,AGT,ACG,GAA,GGA,TCC,AGC,CAG,...,AAA,TCT,TCT,TGC,ATT,CTG,CTG,GGG,CTC,TGA
97,97,CGA,TTC,GTA,CGA,AAA,GAA,CCA,GCC,AGT,...,AAC,CTC,CTG,GCA,TTC,TGA,TGA,GGG,TCA,GAG


In [14]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=3))
ThreeMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    ThreeMer.append(value)
Codon = dict(zip(ThreeMer, [0]*64))

In [15]:
List = ["AGA","GAC","ACT","GGA","TTT","CAA","ATA","AAA","GTA","CCA",
        "GAA","TAT","GCA","TTA","CTA","TCA","ATT","AGT","CAT","ACA"]

In [16]:
# List = ["AGA","GAC","ACT","GGA","TTT","CAA","ATA","AAA","GTA","CCA"]

In [17]:
for i in range(len(List)):
    motif = List[i]
    Codon[motif] = 1

In [18]:
Enhancer = df
Enhancer = Enhancer.drop(["Index"], axis=1)
Enhancer.replace(Codon, inplace=True)
Enhancer["Count"] =  Enhancer.apply(lambda x:x.sum(),axis =1)
Enhancer = Enhancer[["Count"]]
Enhancer

,Count
0,331
1,321
2,341
3,343
4,280
...,...
94,316
95,359
96,319
97,354


In [19]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=3))
ThreeMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    ThreeMer.append(value)
Codon = dict(zip(ThreeMer, [0]*64))

In [20]:
List = ["GAT","ATC","ACC","TTC","CGT","CAG","GTC","TCC","AAG","GGT",
        "CGC","CGG","GAG","TAC","CTC","ACG","GGC","GCC","CCC","CCG"]

In [21]:
for i in range(len(List)):
    motif = List[i]
    Codon[motif] = 1

In [22]:
Silencer = df
Silencer = Silencer.drop(["Index"], axis=1)
Silencer.replace(Codon, inplace=True)
Silencer["Count"] =  Silencer.apply(lambda x:x.sum(),axis =1)
Silencer = Silencer[["Count"]]
Silencer

,Count
0,265
1,286
2,253
3,258
4,311
...,...
94,320
95,229
96,280
97,268


In [23]:
Enhancer.to_csv("EnhancerPosition_High.csv")
Silencer.to_csv("SilencerPosition_High.csv")